# Generation of Habitat and Landcover Layers and Resistance Dictionaries

This notebook generates the habitat layer, landcover layer, and resistance dictionary for each specified bird species based on data from the IUCN Red List and eBird, which are all needed as inputs to the main model. It uses the `ecoscape_layers` package; more information about usage and parameters can be found at https://github.com/ecoscape-earth/ecoscape-layers.

At minimum, you will need:
- API keys for:
    - [the IUCN Red List](http://apiv3.iucnredlist.org/)
    - [eBird](https://science.ebird.org/en/status-and-trends/download-data) (no need for the R package, just request an access key)
- The 6-letter eBird code(s) for the bird species you are creating layers for. You can obtain these codes by looking up bird species on eBird and taking the last 6 letters of the species page's URL (such as "acowoo" from https://ebird.org/species/acowoo).
- An initial landcover map. For EcoScape, we derive our map from the map `iucn_habitatclassification_composite_lvl2_ver004.zip` (cropped to the United States, which is our broad area of study) provided by [Jung et al.](https://zenodo.org/record/4058819).

In [1]:
import os
import shutil
import math
import rasterio.warp
from rasterio import features
from rasterio.crs import CRS
from shapely.geometry import box
from ecoscape_layers import LayerGenerator, warp, reproject_shapefile, make_dirs_for_file
from scgt import *

In [2]:
# Parent folders for data and inputs.
DATA_PATH = "./data"
INPUT_PATH = "./inputs"

# API keys.
REDLIST_KEY = input("IUCN Red List API key: ")
EBIRD_KEY = input("EBird API key: ")

# Global Jung et al. landcover layer:
# https://zenodo.org/records/4058819
landcover_fn = os.path.join(INPUT_PATH, "iucn_habitatclassification_composite_lvl2_ver004.tif")

# Shapefile of US state boundaries, sourced from the U.S. Census Bureau:
# https://catalog.data.gov/dataset/tiger-line-shapefile-current-nation-u-s-state-and-equivalent-entities
# We use this to help obtain bounds for each state.
states_fn = os.path.join(INPUT_PATH, "tl_2023_us_state")

The given parameters specify the parameters used for the EcoScape paper results. You can change them as you see fit.

In [3]:
# Web Mercator projection.
crs = '''
PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]
'''

# Resample using nearest neighbor.
resampling = "near"

# Keep a 100 km padding around the state bounds.
padding = 100000

In [4]:
# State shapes reprojected to Web Mercator.
states_shapes = reproject_shapefile(states_fn, crs)

# Species and states for which we want to compute layers.
species_states = {
    'acowoo': ['AZ', 'CA', 'OR', 'NV', 'NM'],
    'stejay': ['AZ', 'CA', 'UT', 'CO', 'OR', 'MT', 'ID', 'WY', 'NV', 'NM', 'WA'],
    'pilwoo': ['NC', 'TN', 'AR', 'SC', 'GA', 'AL', 'MS', 'CA', 'OR', 'WA', 'ID', 'MT', 'ND', 'MN', 'WI', 'MI', 'NY', 'VT', 'ME', 'MA', 'PA', 'VA', 'WV', 'OH', 'IN', 'IL', 'IA', 'OK', 'TX', 'KS', 'MO', 'LA', 'FL'],
}

# We invert the above dict to map states to species, which is nicer for generating layers by state.
states_species = {}
for k,v in species_states.items():
    for x in v:
        states_species.setdefault(x, []).append(k)

In [ ]:
# We iterate over each state and the state's species for which layers should be generated.
for state, species_codes in states_species.items():
    # We generate the terrain for this state just once in this path,
    # and then copy it to the "{species}/{state}" folders as needed, e.g. to "acowoo/US-CA".
    out_landcover_fn = os.path.join(DATA_PATH, "Terrain", f"terrain_{state}.tif")

    # We get the bounds of the layers for the state based off of the state shape.
    bounds = None
    for s in states_shapes:
        if s['properties']['STUSPS'] == state:
            bounds = features.bounds(s['geometry'])
            break
    if bounds is None:
        raise TypeError(f"Could not find bounds for state {state}.")

    # The above bounds are found in Web Mercator.
    # Mercator is a conformal projection where area/distance iss not preserved;
    # rather, locations with latitude lat are expanded by a factor 1/cos(lat).
    # So, to get a local resolution of around 300m, we convert the bounds to latitude
    # and longitude and compute resolution as 300 / cos((lat_max + lat_min) / 2).
    latlon_bounds = rasterio.warp.transform_geom(CRS.from_string(crs), CRS.from_epsg(4326), box(*bounds))
    resolution = 300 / math.cos(math.radians((latlon_bounds['coordinates'][0][0][1] + latlon_bounds['coordinates'][0][1][1]) / 2))

    # Make directories leading up to the terrain file location if needed,
    # and then process the terrain layer so that it is projected and cropped as we want it.
    make_dirs_for_file(out_landcover_fn)
    warp(landcover_fn, out_landcover_fn, crs, resolution, bounds, padding, resampling)
    layer_generator = LayerGenerator(redlist_key=REDLIST_KEY, landcover_fn=out_landcover_fn, ebird_key=EBIRD_KEY)

    # Then, we generate the habitat layers and initial resistance csvs
    # for each species in this state.
    for species_code in species_codes:
        habitat_fn = os.path.join(DATA_PATH, species_code, f"US-{state}", "habitat.tif")
        resistance_dict_fn = os.path.join(DATA_PATH, species_code, f"US-{state}", "resistance.csv")
        range_fn = os.path.join(DATA_PATH, species_code, "range_map_2022.gpkg")
        range_src = "ebird"
        refine_method = "forest_add308" if species_code == "acowoo" else "forest"

        layer_generator.generate_habitat(species_code, habitat_fn, resistance_dict_fn, range_fn, range_src, refine_method)
        shutil.copy(out_landcover_fn, os.path.join(DATA_PATH, species_code, f"US-{state}", "terrain.tif"))
        shutil.copy(out_landcover_fn + ".aux.xml", os.path.join(DATA_PATH, species_code, f"US-{state}", "terrain.tif.aux.xml"))